# `barusini.tabular`
This notebook provides a walk-through of the `barusini.tabular` python package, which is meant to help with creating ML Pipelines combining advanced feature engineering and modeling techniques for tabular data. 

**The following topics are covered:**


- 1. [Introduction](#intro)<br>

- 2. [Custom ML Pipeline](#custom)<br>

- 3. [Feature Engineering Search](#fe)<br>

- 4. [Model Hyper-Parameter Tunning](#hp)<br>

- 5. [Ensembling](#ens)<br>

- 6. [Model Comparison](#cmp)<br>


# 1. Introduction<a id='intro'></a>

Creating ML models or pipelines consists of multiple stages, exploratory data analysis, data preprocessing, feature engineering, model hyperparameter tuning and potentially ensembling. There are many python libraries that help with one or more of these tasks, however, often applying some of the advanced ML techniques requires writing lot of boilerplate code. 

This package aims to help with data preprocessing, feature engineering, hyperparameter tuning and ensembling, leveraging known python packages that are great at doing what they are supposed to do. It works with `pandas.DataFrame` and `pandas.Series` objects, which are always expected to have proper column names. This makes everything easier to understand and debug. For validation, metric computation and even modeling, `scikit-learn` does most of the heavy lifting. For modeling, any python package that has scikit-learn like API (`fit`/ `predict`/`predict_proba`) can be used, this notebook also uses packages `xgboost` and `lightgbm` to demonstrate the compatibility. For hyper-parameter search, `optuna` is used.

For proper implementation and validation of some advanced ML techniques (like mean target encoding) data preprocessing/feature engineering and modeling have to be bundled in a single object, which makes sure the model fitting and validation are both done properly and are not leaking any target information (overfitting). In the context of this package it is called pipeline.
## 1.1 Pipeline

ML Pipelines are defined by the `Pipeline` object. Pipelines consists of 2 main components:
- **transformers** - defining data preprocessing/feature engineering
- **model** - defines the ML algorithm with it's hyper-parameters

## 1.2 Transformers
Transformers define the data transformations that will be applied to the input data. They are applied serially in the order in which they are defined. Transformers are always initialized with keyword `used_cols=cols_to_use` where `cols_to_use` is a list of one or more columns that are either part of the original data, or are created during applying transformers defined previously in a list.


In this example we are using following transformers:

- `Identity` - take one or more columns from the data and use them as they are (select this variable for a model)
- `CustomOneHotEncoder` - one hot encode one or more input columns, the vector size will be the number of all unique combinations of input column values that appear in the data
- `CustomLabelEncoder` - label encode one or more input columns, it will have as many different values as there are unique combinations of input column values that appear in the data
- `MeanTargetEncoder` - encode one or more input columns by their mean response/target/label value, it is done in CV-fashion to avoid leaking the target

## 1.3 Model

Model is your standard scikit-learn API like model, such as sklearn Random Forest, xgboost, lightgbm and so on. This library does not implement any models, but should be compatible with any other library with scikit-learn like API where models implement these functions:

- `fit` - fitting the model
- `predict` - predicts the hard label
- `predict_proba` - predicts the soft label (probabilities) for classification models


# 2. Custom ML Pipeline<a id='custom'></a>

In this chapter, we will see how a custom ML Pipeline can be defined, validated, fitted, evaluated, used to transform datasets and how we can see the most important features of the Pipeline.

## 2.1 Imports
In addition to `barusini.tabular` we will also use these known packages:

- **copy** - used to copy/clone our models and pipelines
- **pandas** - pipelines work with pandas DataFrames and Series
- **numpy** - used for numeric operations
- **sklearn** - used for validation, metric computation and modeling
- **xgboost** - used for modeling
- **lightgbm** - used for modeling
- **optuna** - is not imported, but it used internally


In [1]:
import copy
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

from barusini.tabular import feature_engineering, model_search, auto_ml
from barusini.tabular.stages.hyper_parameter_tuning import LOG, LOGINT, UNIFORM
from barusini.tabular.transformers import CustomLabelEncoder, CustomOneHotEncoder, MeanTargetEncoder, Pipeline, Ensemble, Identity

from barusini.model_selection import cross_val_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

## 2.2 Loading The Data

In this notebook we will be using Adult dataset https://archive.ics.uci.edu/ml/datasets/adult where the goal is to predict if people's early income exceeds $50 000. The dataset is already split into training and test. We will create following variables:

- `target` - name of the target/label column
- `train_X` - pd.DataFrame, training split without the target column
- `train_y` - pd.Series, training target
- `test_X` - pd.DataFrame, test split without the target column
- `test_y` - pd.Series, test target


In practice we don't need to drop the target column from training/test data frames when using `barusini`, however, it is a good practice to work with the training data without the label to avoid unintentional target leaks in the future.

In [2]:
columns = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "target",
]

train_data = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
train = pd.read_csv(train_data, header=None, names=columns)

test_data = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
test = pd.read_csv(test_data, header=None, names=columns, skiprows=1)

target = "target"
train[target] = train[target].str.contains(">").astype(int)
test[target] = test[target].str.contains(">").astype(int)

train_X = train.drop(columns=[target])
test_X = test.drop(columns=[target])

train_y = train[target]
test_y = test[target]

display(train.head(5))

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


## 2.3 Pipeline Definition

Let's define our first Pipeline. We will use numeric columns `age`,` education-num`, `capital-gain`, `capital-loss` and `hours-per-week` as they are. For string column `marital-status` we will use mean target encoding, for `sex` we will use one-hot encoding and for `workclass` we will use label encoding to demonstrate different encoding techniques. For modeling we will use random forest.


In [3]:
rf_model = Pipeline(
    transformers=[
        Identity(used_cols=["age"]),
        Identity(used_cols=["education-num"]),
        Identity(used_cols=["capital-gain"]),
        Identity(used_cols=["capital-loss"]),
        Identity(used_cols=["hours-per-week"]),
        MeanTargetEncoder(used_cols=["marital-status"]),
        CustomOneHotEncoder(used_cols=["sex"]),
        CustomLabelEncoder(used_cols=["workclass"]),
    ],
    model = RandomForestClassifier(max_depth=7, random_state=42),
)

print(rf_model)

Pipeline (8 Transformers):
    * Identity: ['age']
    * Identity: ['education-num']
    * Identity: ['capital-gain']
    * Identity: ['capital-loss']
    * Identity: ['hours-per-week']
    * Target encoder for feature '['marital-status']'
          * Unfitted Transformer
    * One Hot Encoder for feature '['sex']':
      	Categories: Unfitted Transformer
    * Label Encoder for feature '['workclass']':
      	Categories: Unfitted Transformer
    * RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)



## 2.4 (Cross) Validation
`barusini` implements convenient method `cross_val_score` for computing validation scores, that allows the user to pass in scikit-learn validation objects. This example uses `sklearn.model_selection.StratifiedKFold` validation object.

Our method for computing validation scores takes these parameters:
- `model` - `Pipeline` object
- `X` - pd.DataFrame - input data
- `y` - pd.Series - input target
- `cv` - scikit-learn like validation object
- `scoring` - scikit-learn like metric function
- `n_jobs` - int - number of processes to use, we can speed up validation by providing larger number, each validation split can be evaluated in separate process
- `proba` - bool - whether we should use `predict_proba` or `predict` function of the `model`

In this example, we use Area Under Curve for Receiver Operating Characteristic (ROC AUC) score. This score is useful when we don't need the output probabilities to be calibrated but we want our predictions to be ranked/order properly (higher probability really means an event is more likely, but score 0.9 does not necessarily need mean probability of an event is 90%). Baseline ROC AUC (score of random predictions) is 0.5.


In [4]:
rf_cv_auc = cross_val_score(
    model=rf_model,
    X=train_X,
    y=train_y,
    cv=StratifiedKFold(),
    scoring=roc_auc_score,
    n_jobs=1,
    proba=True,
)
print("CV AUC:", round(rf_cv_auc, 4))

CV AUC: 0.9115


## 2.5 Pipeline evaluation

After we estimated what our pipeline performance should be, we can evaluate it on actual test data. We will firstly use pipeline's `fit` method to fit our model on full training data, then `predict_proba` method to predict the probabilities. Since this is a binary classification problem, our model will output vector of size 2 when making predictions, these will be the probabilities of the two classes. For computing ROC AUC, we will only use the probability of event happening (person earning more than 50K). Afterwards, we can print the fitted pipeline and the test score.

In [5]:
rf_model.fit(train_X, train_y)
test_preds = rf_model.predict_proba(test_X)
rf_test_auc = roc_auc_score(test_y, test_preds[:,1])
print(rf_model)
print("Test AUC", round(rf_test_auc, 4))

Pipeline (8 Transformers):
    * Identity: ['age']
    * Identity: ['education-num']
    * Identity: ['capital-gain']
    * Identity: ['capital-loss']
    * Identity: ['hours-per-week']
    * Target encoder for feature '['marital-status']'
          * Pipeline(memory=None,
                     steps=[('enc',
                             OneHotEncoder(categories='auto', drop=None,
                                           dtype=<class 'numpy.float64'>,
                                           handle_unknown='ignore', sparse=True)),
                            ('mean',
                             LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                              normalize=False))],
                     verbose=False)
    * One Hot Encoder for feature '['sex']':
      	Categories: [' Female' ' Male']
    * Label Encoder for feature '['workclass']':
      	Categories: [' ?' ' Federal-gov' ' Local-gov' ' Never-worked' ' Private'
       ' Self-em

## 2.6 Show Pipeline Transformations
After we fit our pipeline, we can easily show the original dataset along with the new transformed features, that get created by our pipeline

In [6]:
rf_model.transform(test_X).head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,marital-status _TE_,sex _OHE_ Female_,sex _OHE_ Male_,workclass _LE_
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0.045961,0.0,1.0,4
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0.446848,0.0,1.0,4
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0.446848,0.0,1.0,2
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,0.446848,0.0,1.0,4
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0.045961,1.0,0.0,0


## 2.7 Show only features used by the model

We can also subset transformed dataset to only features used by the model.

In [7]:
rf_model.transform(test_X)[rf_model.used_cols].head(5)

,age,capital-gain,capital-loss,education-num,hours-per-week,marital-status _TE_,sex _OHE_ Female_,sex _OHE_ Male_,workclass _LE_
0,25,0,0,7,40,0.045961,0.0,1.0,4
1,38,0,0,9,50,0.446848,0.0,1.0,4
2,28,0,0,12,40,0.446848,0.0,1.0,2
3,44,7688,0,10,40,0.446848,0.0,1.0,4
4,18,0,0,10,30,0.045961,1.0,0.0,0


## 2.8 Show model feature importance

There is also a possibility to show the feature importance of our model by calling it's `varimp` method.

In [8]:
rf_model.varimp()

,Importance
Feature,
marital-status _TE_,1.000000
capital-gain,0.828309
education-num,0.578073
age,0.298391
capital-loss,0.172780
hours-per-week,0.152922
sex _OHE_ Male_,0.050922
sex _OHE_ Female_,0.040938
workclass _LE_,0.015075


# 3. Feature Engineering Search<a id='fe'></a>

In practice we might not know what the data processing pipeline should be, we want the one that optimizes our selected metric the most for a given model/algorithm. For that reason `barusini.tabular` implements function `feature_engineering` that aims to find such data pipeline.

The parameters of this function are:

- `X` - pd.DataFrame - input data
- `y` - pd.Series - input target
- `model_path` - optional string - if provided, the pipeline will be saved as a pickle to this path
- `imputation_stage` - bool - whether we allow imputation of missing values
- `subset_stage` - bool - whether we allow subsetting unimportant features
- `encode_stage` - bool - whether we allow encoding string/text columns
- `recode_stage` - bool - whether we allow encoding numeric columns and treating them as categorical
- `allowed_transformers` - list of transformers - what are the transformers that can be used by encode/recode stages
- `estimator` - scikit-learn API compatible model - model that will be used for predictions
- `cv` - scikit-learn like validation object
- `metric` -  scikit-learn like metric function
- `maximize` - bool - whether the `metric` should be maximized (True) or minimized (False)
- `proba` -  bool - whether we should use predict_proba or predict function of the model
 
This function finds the best pipeline for a given model in a deterministic and reproducible way, as long as the model and the transformers allowed are all reproducible. In this example we are searching for the optimal data pipeline for xgboost model.

In [9]:
xgb_model = feature_engineering(
    X=train_X,
    y=train_y,
    model_path=None,  # do not save model to pickle file
    imputation_stage=True,  # try to impute missing valies
    subset_stage=True,  # try subseting input features
    encode_stage=True,  # try encoding categorical features
    recode_stage=True,  # try recoding numeric features
    allowed_transformers=(  # allow only basic categorical encoders for encode/recode stages
        CustomLabelEncoder,
        CustomLabelEncoder,
        MeanTargetEncoder,
    ),
    estimator=XGBClassifier(use_label_encoder=False, eval_metric="logloss"),  # use xgboost model
    cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),  # make stratified CV
    metric=roc_auc_score,  # optimize AUC
    classification=True,  # problem is classification
    maximize=True,  # maximize metric (AUC)
    proba=True,  # use soft predictions for metric (AUC)
)

Duration of stage Basic Preprocessing: 1.442262887954712 seconds
Duration of stage Find Best Imputation: 0.0021479129791259766 seconds
-------------------------------- Starting stage Finding best subset --------------------------------
BASE 0.8714933333635003


CURRENT BEST 0.8752087294385822


ORIGINAL BEST 0.8714933333635003
NEW BEST 0.8752087294385822
DIFF 0.003715396075081867
Dropped ['fnlwgt']
New []
-------------------------------- Stage Finding best subset finished --------------------------------
Duration of stage Find Best Subset: 27.462732076644897 seconds
Encoding stage for  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')


  0%|          | 0/8 [00:00<?, ?it/s]

Encoders for workclass: ['MeanTargetEncoder', 'CustomLabelEncoder']
-------------------------- Starting stage Encoding categoricals workclass --------------------------
BASE 0.8752087294385822


...

  0%|          | 0/1 [00:00<?, ?it/s]

Encoders for education-num: ['MeanTargetEncoder']
------------------------------- Starting stage Recoding education-num -------------------------------
BASE 0.9268893729600253



100%|██████████| 1/1 [00:03<00:00,  3.57s/it]
                                             

CURRENT BEST 0.9270039949140232
ORIGINAL BEST 0.9268893729600253
NEW BEST 0.9270039949140232
DIFF 0.00011462195399791764
Dropped ['education-num']
New ['education-num _TE_']
------------------------------- Stage Recoding education-num finished -------------------------------


Duration of stage Recode categoricals: 9.29783296585083 seconds
Final features: ['age', 'capital-gain', 'capital-loss', 'education', 'education-num', 'education-num _TE_', 'fnlwgt', 'hours-per-week', 'marital-status', 'marital-status _TE_', 'native-country', 'occupation', 'occupation _TE_', 'race', 'race _TE_', 'relationship', 'relationship _LE_', 'sex', 'sex _LE_', 'workclass', 'workclass _LE_']
Duration of stage Feature engineering: 1.6880293806393942 minutes


## 3.1 Cross-Validation

In [10]:
xgb_cv_auc = cross_val_score(
    model=xgb_model,
    X=train_X,
    y=train_y,
    cv=StratifiedKFold(),
    scoring=roc_auc_score,
    n_jobs=1,
    proba=True,
)
print("CV AUC:", round(xgb_cv_auc, 4))

CV AUC: 0.9264


## 3.2 Evaluate The Pipeline

Here we define the `pipeline_summary` function printing out the Pipeline along with the test ROC AUC score.

In [11]:
def pipeline_summary(pipeline, test_X, test_y, verbose=True):
    test_preds = pipeline.predict_proba(test_X)
    test_auc = roc_auc_score(test_y, test_preds[:,1])
    if verbose:
        print(pipeline)
        print("Test AUC:", round(test_auc, 4))
    return test_auc

xgb_test_auc = pipeline_summary(xgb_model, test_X, test_y)

Pipeline (11 Transformers):
    * Identity: ['age']
    * Identity: ['capital-gain']
    * Identity: ['capital-loss']
    * Identity: ['hours-per-week']
    * Label Encoder for feature '['workclass']':
      	Categories: [' ?' ' Federal-gov' ' Local-gov' ' Never-worked' ' Private'
       ' Self-emp-inc' ' Self-emp-not-inc' ' State-gov' ' Without-pay']
    * Target encoder for feature '['marital-status']'
          * Pipeline(memory=None,
                     steps=[('enc',
                             OneHotEncoder(categories='auto', drop=None,
                                           dtype=<class 'numpy.float64'>,
                                           handle_unknown='ignore', sparse=True)),
                            ('mean',
                             LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                              normalize=False))],
                     verbose=False)
    * Target encoder for feature '['occupation']'
          *

# 4. Tune Hyper-Parameters of Pipeline's Model<a id='hp'></a>

First, we will copy our xgboost pipeline, keep the data processing the same and replace the model with a lightgbm model. Then we will use Pipeline's `tune` method to tune the hyper-parameters of the Pipeline's model. Note that calling the `tune` function changes the model and also refit's the Pipeline.

The `tune` function takes following arguments:

- `X` - pd.DataFrame - input data
- `y` - pd.Series - input target
- `cv` - scikit-learn like validation object
- `score` -  scikit-learn like metric function
- `maximize` - bool - whether the `metric` should be maximized (True) or minimized (False)
- `probability` -  bool - whether we should use predict_proba or predict function of the model
- `static_params` - dict - static/fixed hyper-parameters
- `params` - dict - hyper-parameters to tune, these are defined as following:
    - *key* - name of the hyper-parameter
    - *value* - tuple with 3 entries
        - distribution type (LOG, LOGINT, UNIFORM)
        - min possible value
        - max possible value
- `n_trials` - int - number of different combinations tried
- `n_jobs` - int - number of processes used
- `seed` - optional int - seed to make tuning reproducible
 
After tuning the Pipeline, we will print it out together with the test ROC AUC score.


In [12]:
lgb_model = copy.deepcopy(xgb_model)
lgb_model.model = LGBMClassifier(random_state=42)

lgb_model.tune(
    X=train_X,
    y=train_y,
    cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
    score=roc_auc_score,
    probability=True,
    maximize=True,
    static_params={"n_estimators": 100, "seed": 42, "n_jobs": 1},
    # hyper parameters and tuning space
    params={
        "min_child_samples": (LOGINT, (1, 1000)),
        "num_leaves": (LOGINT, (2 ** 3, 2 ** 12)),
        "learning_rate": (LOG, (1e-4, 1e1)),
        "subsample": (UNIFORM, (0.6, 1)),
        "colsample_bytree": (UNIFORM, (0.6, 1)),
    },
    n_trials=20,
    n_jobs=1,  # make tuning deterministic
    seed=42,  # make tuning deterministic
)

lgb_cv_auc = cross_val_score(
    model=lgb_model,
    X=train_X,
    y=train_y,
    cv=StratifiedKFold(),
    scoring=roc_auc_score,
    n_jobs=1,
    proba=True,
)

lgb_test_auc = pipeline_summary(lgb_model, test_X, test_y)
print("CV AUC:", round(lgb_cv_auc, 4))

[I 2021-08-09 17:03:23,375] A new study created in memory with name: no-name-bcdc4797-d8d0-4492-aef6-f7c052c7eac7
[I 2021-08-09 17:03:37,741] Trial 0 finished with value: -0.9068281601275183 and parameters: {'min_child_samples': 9, 'num_leaves': 3003, 'learning_rate': 0.4570563099801455, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'n_estimators': 100}. Best is trial 0 with value: -0.9068281601275183.
[I 2021-08-09 17:03:39,878] Trial 1 finished with value: -0.6012670386843035 and parameters: {'min_child_samples': 2, 'num_leaves': 11, 'learning_rate': 2.1423021757741068, 'subsample': 0.8404460046972835, 'colsample_bytree': 0.8832290311184181, 'n_estimators': 100}. Best is trial 0 with value: -0.9068281601275183.
[I 2021-08-09 17:03:44,631] Trial 2 finished with value: -0.6735121719333567 and parameters: {'min_child_samples': 1, 'num_leaves': 3389, 'learning_rate': 1.452824663751602, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.6727299868828402, 'n_

Out of 20 trials the best score is -0.9271149539038224 with params {'min_child_samples': 2, 'num_leaves': 51, 'learning_rate': 0.042051564509138675, 'subsample': 0.7727780074568463, 'colsample_bytree': 0.7164916560792167, 'n_estimators': 100}
Finished loading model, total used 100 iterations
Finished loading model, total used 100 iterations
Finished loading model, total used 100 iterations
Finished loading model, total used 100 iterations
Finished loading model, total used 100 iterations
Pipeline (11 Transformers):
    * Identity: ['age']
    * Identity: ['capital-gain']
    * Identity: ['capital-loss']
    * Identity: ['hours-per-week']
    * Label Encoder for feature '['workclass']':
      	Categories: [' ?' ' Federal-gov' ' Local-gov' ' Never-worked' ' Private'
       ' Self-emp-inc' ' Self-emp-not-inc' ' State-gov' ' Without-pay']
    * Target encoder for feature '['marital-status']'
          * Pipeline(memory=None,
                     steps=[('enc',
                             

# 5. Ensembling Multiple Pipelines<a id='ens'></a>

We can define multiple Pipelines, fit each of them individually and then create additional 2-nd level model (meta model) taking all of their predictions as input generating new predictions to achieve better performance. This process is called ensembling and can be done by the `Ensemble` object.

`Ensemble` object is defined by providing:
- `pipelines` - list of Pipelines to use as 1-st level models
- `meta` - optional model to use as 2-nd level model - if it is not provided it will be a simple weighted average where all weights are positive and sum to 1
- `cv` - scikit-learn like validation object that defines how the Out-Of-Fold (OOF) predictions of 1-st level models will be generated to create features for 2-nd level model

The ensemble object can be fitted or (cross) validated just like a regular pipeline. Since fitting an ensemble object requires generating OOF predictions, some form of validation is done during fitting, so computing cross-validation score of an ensemble object results in nested (cross) validation, therefor it can take some time.

In [13]:
ensemble = Ensemble(pipelines=[rf_model, xgb_model, lgb_model])
ensemble_cv_auc = cross_val_score(
    model=ensemble,
    X=train_X,
    y=train_y,
    cv=StratifiedKFold(),
    scoring=roc_auc_score,
    n_jobs=1,
    proba=True,
)

ensemble.fit(train_X, train_y)
ensemble_test_auc = pipeline_summary(ensemble, test_X, test_y)
print("Ensemble CV AUC:", round(ensemble_cv_auc, 4))

Finished loading model, total used 100 iterations


100%|██████████| 3/3 [00:00<00:00, 8224.13it/s]


Finished loading model, total used 100 iterations


100%|██████████| 3/3 [00:00<00:00, 158.58it/s]


Finished loading model, total used 100 iterations


...

Finished loading model, total used 100 iterations


100%|██████████| 3/3 [00:00<00:00, 161.47it/s]


Finished loading model, total used 100 iterations


100%|██████████| 3/3 [00:00<00:00, 11586.48it/s]


Ensemble,  (WeightedAverage, 3 Pipelines):
    * Pipeline (8 Transformers):
          * Identity: ['age']
          * Identity: ['education-num']
          * Identity: ['capital-gain']
          * Identity: ['capital-loss']
          * Identity: ['hours-per-week']
          * Target encoder for feature '['marital-status']'
                * Pipeline(memory=None,
                           steps=[('enc',
                                   OneHotEncoder(categories='auto', drop=None,
                                                 dtype=<class 'numpy.float64'>,
                                                 handle_unknown='ignore', sparse=True)),
                                  ('mean',
                                   LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                                    normalize=False))],
                           verbose=False)
          * One Hot Encoder for feature '['sex']':
            	Categories: [' Female' ' M

## 5.1 Show Ensemble Weights

If we use default enseble meta algorithm (simple weighted average) we can see the weights of the individual Pipelines. We can also verify that the weights sum up to 1 and are non-negative.

In [14]:
assert np.isclose(ensemble.meta.weights.sum(), 1)
assert all(ensemble.meta.weights >= 0)
ensemble.meta.weights

array([0.0417442 , 0.52406773, 0.43418808])

# 6. Model Comparison<a id='cmp'></a>

Finally we can compare the cross-validation and test performance of the pipelines we created and their ensemble. We can see that the ensemble achieves better performance, it does however consist of three individual pipelines so it is more complex and slower.

In [15]:
scores_data = {
    "model": ["Random Forest", "XGBoost", "LightGBM", "Ensemble"],
    "CV AUC": [rf_cv_auc, xgb_cv_auc, lgb_cv_auc, ensemble_cv_auc],
    "Test AUC": [rf_test_auc, xgb_test_auc, lgb_test_auc, ensemble_test_auc],
}

scores = pd.DataFrame(scores_data).sort_values("Test AUC", ascending=False)
scores["rank"] = range(1, scores.shape[0]+1)
scores

,model,CV AUC,Test AUC,rank
3,Ensemble,0.928231,0.927614,1
2,LightGBM,0.927467,0.926760,2
1,XGBoost,0.926387,0.926413,3
0,Random Forest,0.911471,0.910122,4
